In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

# from itertools import starmap
from utils import save_model, save_plots
from CNN_execution import plot_roc_curve, ect_train_validate, report_trained_model, find_numpy_files

In [ ]:
# Parameters required to define the model. 
# Will remain same throught the excerise.
 
NUM_EPOCHS = 10 # number of epochs to train the network for; type=int
LEARNING_RATE = 1e-3 # learning rate for training; type=float
# loss function
lossfcn = nn.CrossEntropyLoss()

# Number of workers for dataloader
num_workers = int( os.environ.get('SLURM_CPUS_PER_TASK', default=0) ) - 2
num_workers = 16 if num_workers < 0 else num_workers

# device
device = ('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
data_dir = '../../data'
classes = [
    i
        for i in os.listdir(data_dir)
        if os.path.isdir(os.path.join(data_dir, i))
]
class_items = {
    i: find_numpy_files(os.path.join(data_dir, i))
        for i in classes
}
class_items.pop('Transect')
class_items.pop('Leafsnap')

num_data_to_use_for_training = min( [len(class_items[i]) for i in class_items] )
# num_data_to_use_for_training = 64
print(f"Using {num_data_to_use_for_training} data for training")

class_items = {
    class_name: np.random.choice( file_paths, num_data_to_use_for_training, replace=False)
        for class_name, file_paths in class_items.items()
}

Using 865 data for training


In [ ]:
for i in range(8,11):
    i = 2**i
    directions , thresholds = int(i), int(i)
    trained_model = ect_train_validate(
        num_dirs=directions,
        num_thresh=thresholds,
        input_path=class_items,
        output_ect_path=f'outputs/output_{directions}_{thresholds}/ect',
        output_model_path=f'outputs/output_{directions}_{thresholds}/best_model.pth',
        num_workers=num_workers,
        parallel=True,
        batch_size=4,
        num_epochs=NUM_EPOCHS,
        log_level='None'
    )
    save_model(
        epochs=trained_model["num_epochs"],
        model=trained_model["model"],
        optimizer=trained_model["optimizer"],
        criterion=trained_model["lossfcn"],
        output_model_path=f'outputs/output_{directions}_{thresholds}/best_model.pth',
    )
    loss, acc = plt.figure(figsize=(9,5)).subplots(1, 2)
    save_plots(
        train_acc= trained_model["train_acc"],
        valid_acc= trained_model["valid_acc"],
        train_loss= trained_model["train_loss"],
        valid_loss= trained_model["valid_loss"],
        loss=loss,
        accuracy=acc,
        accuracy_path=f'outputs/output_{directions}_{thresholds}/accuracy_loss.png',
        loss_path=f'outputs/output_{directions}_{thresholds}/accuracy_loss.png'
    )
    ax = plt.figure( figsize=(24,24), dpi=300 ).add_subplot(111)
    report_trained_model(
        num_dirs=directions,
        num_thresh=thresholds,
        train_dataset=trained_model["train_dataset"],
        train_loader=trained_model["train_loader"],
        test_loader=trained_model["test_loader"],
        test_dataset=trained_model["test_dataset"],
        ax=ax,
        model_path=f'outputs/output_{directions}_{thresholds}/best_model.pth',
        output_cf=f'outputs/output_{directions}_{thresholds}/confusion_matrix.png',
        output_report=f'outputs/output_{directions}_{thresholds}/accuracy.txt',
        log_level='None'
    )
    plot_roc_curve(
        model=trained_model["model"],
        test_loader=trained_model["test_loader"],
        test_dataset=trained_model["test_dataset"],
        output_path=f'outputs/output_{directions}_{thresholds}/roc_curve.png'
    )
    print(f"Completed training for {directions} directions and {thresholds} thresholds")

Computing ECT dataset by clearing out the output directory
Saving final model...


/mnt/ufs18/rs-037/HRT841_FS24_001/ECT-Leaf-CNN/leaf-example-tutorial/CNN_execution.py:234: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_path)[

Using validation to compute ROC curve


/mnt/home/karkinis/.conda/envs/ECT_CNN.cpu/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/mnt/home/karkinis/.conda/envs/ECT_CNN.cpu/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/mnt/home/karkinis/.conda/envs/ECT_CNN.cpu/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

Completed training for 4 directions and 4 thresholds
Computing ECT dataset by clearing out the output directory
Saving final model...


...

/mnt/ufs18/rs-037/HRT841_FS24_001/ECT-Leaf-CNN/leaf-example-tutorial/CNN_execution.py:234: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_path)[

Using validation to compute ROC curve


100%|██████████| 605/605 [00:02<00:00, 252.20it/s]


Completed training for 64 directions and 64 thresholds
Computing ECT dataset by clearing out the output directory


/mnt/ufs18/rs-037/HRT841_FS24_001/ECT-Leaf-CNN/leaf-example-tutorial/CNN_execution.py:355: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, axes = plt.subplots(x,y)
